In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [27]:
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [3]:
numeros = pd.read_csv("classificacao_Q4.csv")

In [4]:
numeros.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,target
0,1.696199,-0.792598,-0.349427,-0.464560,3.187014,0.035976,1.033274,-1.504968,0.204693,1.691204,...,1.488142,-0.686337,2.084970,-0.685140,-2.049451,2.015426,1.158477,-0.309441,-1.549833,4.0
1,-0.236696,-2.202342,0.024023,1.497700,-0.069758,-2.467088,1.126529,-0.570557,2.079251,-1.882632,...,0.405567,0.509564,1.374071,-0.016943,-0.429280,-0.895016,1.259566,-0.354139,0.806797,5.0
2,-0.436683,1.563816,-0.895999,-0.580425,0.311060,-0.187369,0.805249,-2.399522,-0.578818,1.586981,...,0.933578,-1.285978,0.503162,0.204829,-0.753835,0.290033,1.721487,1.304518,0.478903,3.0
3,1.425908,0.400055,-0.305038,-0.930251,-2.214549,1.763379,-0.239868,-2.058891,-1.006533,-2.156839,...,-0.849927,1.402768,0.393653,-1.466818,0.152257,-4.004950,0.676342,-1.927319,1.959032,8.0
4,-0.186156,-0.975764,0.594660,-1.181980,-1.443414,-0.797651,-1.252608,-0.060452,0.130702,-2.343517,...,-1.444435,-1.818126,0.446574,0.239328,0.802939,-2.035289,-1.433793,-0.218596,0.619317,9.0


In [5]:
numeros.shape

(1500, 101)

In [6]:
numeros_x = pd.DataFrame()

In [7]:
numeros_y = pd.DataFrame()

In [8]:
numeros_x = numeros.iloc[:,:-1]
print(numeros_x.head())
print(numeros_x.shape)


numeros_y = numeros.iloc[:,-1:]
print(numeros_y.head())
print(numeros_y.shape)

         x0        x1        x2        x3        x4        x5        x6  \
0  1.696199 -0.792598 -0.349427 -0.464560  3.187014  0.035976  1.033274   
1 -0.236696 -2.202342  0.024023  1.497700 -0.069758 -2.467088  1.126529   
2 -0.436683  1.563816 -0.895999 -0.580425  0.311060 -0.187369  0.805249   
3  1.425908  0.400055 -0.305038 -0.930251 -2.214549  1.763379 -0.239868   
4 -0.186156 -0.975764  0.594660 -1.181980 -1.443414 -0.797651 -1.252608   

         x7        x8        x9    ...          x90       x91       x92  \
0 -1.504968  0.204693  1.691204    ...    -0.534360  1.488142 -0.686337   
1 -0.570557  2.079251 -1.882632    ...     0.498207  0.405567  0.509564   
2 -2.399522 -0.578818  1.586981    ...    -2.630024  0.933578 -1.285978   
3 -2.058891 -1.006533 -2.156839    ...    -0.260665 -0.849927  1.402768   
4 -0.060452  0.130702 -2.343517    ...     1.098563 -1.444435 -1.818126   

        x93       x94       x95       x96       x97       x98       x99  
0  2.084970 -0.685140 -2

In [2]:
def cross_validate_knn(cv,data_x,data_y,neighbors,criterio='Euclidean',model_type='Classifier'):
    scores_train= []
    scores_test= []

    acc_train = []
    acc_test = []
    #auc_train = []
    #auc_test = []
    precision_test = []
    precision_train = []
    recall_test = []
    recall_train = []
    f1_test = []
    f1_train = []
    
    y_pred_train = []
    y_pred_test = []
    mse_train = []
    rmse_train = []
    mse_test = []
    rmse_test = []
    mae_train = []
    rmae_train = []
    mae_test = []
    rmae_test = []
    
    for train_idx, test_idx in cv.split(data_x):
        X_train, X_test = data_x.iloc[train_idx], data_x.iloc[test_idx] 
        y_train, y_test = data_y.iloc[train_idx], data_y.iloc[test_idx]
        #print(test_idx)
        if model_type=='Classifier':
            knn = KNeighborsClassifier(n_neighbors=neighbors, metric=criterio)
            knn.fit(X_train, y_train)
            y_pred_train = knn.predict(X_train)
            y_pred_test = knn.predict(X_test)
            acc_test.append(accuracy_score(y_test, y_pred_test))
            acc_train.append(accuracy_score(y_train, y_pred_train))
            precision_test.append(precision_score(y_test, y_pred_test,average='micro'))
            precision_train.append(precision_score(y_train, y_pred_train,average='micro'))
            recall_test.append(recall_score(y_test, y_pred_test,average='micro'))
            recall_train.append(recall_score(y_train, y_pred_train,average='micro'))
            f1_test.append(f1_score(y_test, y_pred_test,average='micro'))
            f1_train.append(f1_score(y_train, y_pred_train,average='micro'))
            #auc_train.append(roc_auc_score(y_train, y_pred_train))
            #auc_test.append(roc_auc_score(y_test, y_pred_test))
            
        else:
            knn = KNeighborsRegressor(n_neighbors=neighbors, metric=criterio)
            knn.fit(X_train, y_train)
            y_pred_train = knn.predict(X_train)
            y_pred_test = knn.predict(X_test)
            scores_test.append(knn.score(X_test,y_test))
            scores_train.append(knn.score(X_train,y_train))
            mse_train.append(metrics.mean_squared_error(y_train,y_pred_train))
            rmse_train.append(math.sqrt(metrics.mean_squared_error(y_train,y_pred_train)))
            mse_test.append(metrics.mean_squared_error(y_test,y_pred_test))
            rmse_test.append(math.sqrt(metrics.mean_squared_error(y_test,y_pred_test)))
            mae_train.append(metrics.mean_absolute_error(y_train,y_pred_train))
            rmae_train.append(math.sqrt(metrics.mean_absolute_error(y_train,y_pred_train)))
            mae_test.append(metrics.mean_absolute_error(y_test,y_pred_test))
            rmae_test.append(math.sqrt(metrics.mean_absolute_error(y_test,y_pred_test)))

            
    
    if model_type=='Classifier':
        print("TRAIN :")
        print('Accuracy List Train: ', acc_train)
        print('Accuracy Mean', np.mean(acc_train))
        print('Precision Mean', np.mean(precision_train))
        print('Recall Mean', np.mean(recall_train))
        print('F1 Mean', np.mean(f1_train))
        print("TEST :")
        print('Accuracy List Train: ', acc_test)
        print('Accuracy Mean', np.mean(acc_test))
        print('Precision Mean', np.mean(precision_test))
        print('Recall Mean', np.mean(recall_test))
        print('F1 Mean', np.mean(f1_test))
    else:
        print("TEST :")
        print('R2 Test: ', scores_test)
        print('Media R2 Test Set: ',np.mean(scores_test))
        print('MSE Test: ', mse_test)
        print("Mean MSE: ", np.mean(mse_test))
        print("New RMSE: ", np.mean(rmse_test))
        print('MAE Test: ', mae_test)
        print("Mean MAE: ", np.mean(mae_test))
        print("New RMAE: ", np.mean(rmae_test))

        print("\n")

        print("TRAIN :")
        print('R2 Train: ',scores_train)
        print('Media R2 Train Set: ',np.mean(scores_train))
        print('MSE Train: ', mse_train)
        print("Mean MSE: ", np.mean(mse_train))
        print("New RMSE: ", np.mean(rmse_train))
        print('MAE Train: ', mae_train)
        print("Mean MAE: ", np.mean(mae_train))
        print("New RMAE: ", np.mean(rmae_train))
    #y_pred_test = tree.predict(X_test)
    #y_pred_train = tree.predict(X_train)

    #Score on Classifier is Mean Accuracy , and R2 on Regressor
    
    
    #print("Train: ", train_idx)
    #print("Test: ", test_idx)

In [3]:
knn = KNeighborsClassifier(n_neighbors=15, metric='euclidean')
kf = KFold(n_splits=10,shuffle=False)
cross_validate_knn(kf,numeros_x,numeros_y,neighbors=15,criterio='euclidean',model_type='Classifier')

cval = cross_val_score(knn, numeros_x, numeros_y, cv=kf,scoring='accuracy')
print("Mean cval_score (Test): ", np.mean(cval))

cvalidate = cross_validate(knn,numeros_x,numeros_y,cv=kf,scoring='accuracy', return_train_score=True)
print(sorted(cvalidate.keys()))
print("Train: ",np.mean(cvalidate['train_score']))
print("Test: ",np.mean(cvalidate['test_score']))
#'accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'completeness_score', 
#'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 
#'homogeneity_score', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 
#'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 
#'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 
#'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score'

NameError: name 'KNeighborsClassifier' is not defined

In [25]:
np.mean(scores)

0.6519999999999999

In [21]:
dataset = pd.read_csv('regressao_Q6.csv')
dataset.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,target
0,-0.351837,0.719462,0.862522,-1.131049,0.744903,0.532177,1.595572,0.607174,1.306696,0.450022,170.471696
1,-0.309565,0.786033,1.175365,-1.149287,0.318301,0.620987,1.470917,0.568968,1.334661,-0.267235,151.023999
2,-1.808729,-0.581660,-0.052200,-2.411602,0.171517,-1.174627,0.010317,-0.677372,0.841643,-1.186626,-268.606299
3,-0.386627,0.991724,0.234672,-0.431101,-0.160947,0.521185,1.957080,0.651099,0.933481,1.280705,180.737656
4,-0.879158,0.106556,0.775418,-1.046543,0.097267,-0.757026,1.050900,-0.262257,1.383563,-1.152509,-38.200673


In [22]:
numeros_x2 = pd.DataFrame()
numeros_x2 = dataset.iloc[:,:-1]
print(numeros_x2.head())
print(numeros_x2.shape)

         x0        x1        x2        x3        x4        x5        x6  \
0 -0.351837  0.719462  0.862522 -1.131049  0.744903  0.532177  1.595572   
1 -0.309565  0.786033  1.175365 -1.149287  0.318301  0.620987  1.470917   
2 -1.808729 -0.581660 -0.052200 -2.411602  0.171517 -1.174627  0.010317   
3 -0.386627  0.991724  0.234672 -0.431101 -0.160947  0.521185  1.957080   
4 -0.879158  0.106556  0.775418 -1.046543  0.097267 -0.757026  1.050900   

         x7        x8        x9  
0  0.607174  1.306696  0.450022  
1  0.568968  1.334661 -0.267235  
2 -0.677372  0.841643 -1.186626  
3  0.651099  0.933481  1.280705  
4 -0.262257  1.383563 -1.152509  
(1100, 10)


In [23]:
numeros_y2 = pd.DataFrame()
numeros_y2 = dataset.iloc[:,-1:]
print(numeros_y2.head())
print(numeros_y2.shape)

       target
0  170.471696
1  151.023999
2 -268.606299
3  180.737656
4  -38.200673
(1100, 1)


In [34]:
def rmse(estimator, X, y):
    y_pred = estimator.predict(X)
    return math.sqrt(mean_squared_error(y,y_pred))

def rmae(estimator, X, y):
    y_pred = estimator.predict(X)
    return math.sqrt(mean_absolute_error(y,y_pred))

In [33]:
knn = KNeighborsRegressor(n_neighbors=15, metric='euclidean')
kf = KFold(n_splits=10,shuffle=False)
cross_validate_knn(kf,numeros_x2,numeros_y2,neighbors=15,criterio='euclidean',model_type='Regressor')

cval = cross_val_score(knn, numeros_x, numeros_y, cv=kf,scoring=rmse)
print("Mean cval_score (Test): ", np.mean(cval))

cvalidate = cross_validate(knn,numeros_x,numeros_y,cv=kf,scoring=rmse, return_train_score=True)
print(sorted(cvalidate.keys()))
print("Train: ",np.mean(cvalidate['train_score']))
print("Test: ",np.mean(cvalidate['test_score']))

TEST :
R2 Test:  [0.9696232386587471, 0.9881049273637461, 0.9854340875247855, 0.9890782591341369, 0.9834975489731996, 0.9817793467504767, 0.9854886269788444, 0.9886356820315791, 0.9898760999659663, 0.9854294179236618]
Media R2 Test Set:  0.9846947235305145
MSE Test:  [5365.346038408826, 1871.6564782842925, 2811.862489743019, 1516.7688658716424, 2689.5341920385154, 2665.4347836810616, 2289.8186310910073, 1986.1310129315732, 1622.5262141271908, 2534.81564505712]
Mean MSE:  2535.389435123425
Root Mean MSE:  50.352650725889546
New RMSE:  49.50180758742825
MAE Test:  [45.2151588209282, 32.94054692400442, 39.16294906503819, 29.304145316675932, 35.528333357913645, 38.160844747975574, 37.076652653045926, 33.7236422850446, 31.46755274079339, 37.382295454551986]
Mean MAE:  35.99621213659719
Root Mean MAE:  5.999684336412807
New RMAE:  5.9892946916237975


TRAIN :
R2 Train:  [0.9880249555993436, 0.9866728613931675, 0.9867481925599936, 0.9867868206927772, 0.9865370792627792, 0.9870928759159325, 0.

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
